In [1]:
import os
os.chdir('..')

In [2]:
from ddi_kt_2024.multimodal.model import MultimodalModel
from ddi_kt_2024.utils import get_lookup, load_pkl, get_labels
from ddi_kt_2024.text.model.word_embedding import WordEmbedding
from ddi_kt_2024.text.model.custom_dataset import BertEmbeddingDataset
from ddi_kt_2024.mol.preprocess import mapped_smiles_reader, candidate_smiles
from ddi_kt_2024.mol.mol_dataset import MolDataset
from ddi_kt_2024.multimodal.dataset import MultimodalDataset
from ddi_kt_2024.multimodal.trainer import Trainer
from ddi_kt_2024.mol.gnn import GNN

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-04 03:59:57,436 - INFO - Enabling RDKit 2023.09.5 jupyter extensions
2024-03-04 03:59:58,068 - DEBUG - matplotlib data path: /home/codespace/.local/lib/python3.10/site-packages/matplotlib/mpl-data
2024-03-04 03:59:58,074 - DEBUG - CONFIGDIR=/home/codespace/.config/matplotlib
2024-03-04 03:59:58,077 - DEBUG - interactive is False
2024-03-04 03:59:58,078 - DEBUG - platform is linux
2024-03-04 03:59:58,094 - DEBUG - CACHEDIR=/home/codespace/.cache/matplotlib
2024-03-04 03:59:58,096 - DEBUG - Using fontManager instance from /home/codespace/.cache/matplotlib/fontlist-v330.json
2024-03-04 03:59:59,304 - DEBUG - Popen(['git', 'version'], cwd=/workspaces/DDI-KT-2024, stdin=None, shell=False, universal_newlines=False)
2024-03